**File Location**: `notebooks/01_random_walk.ipynb`

# Random Walk Visualization

## Introduction

This notebook demonstrates the implementation and visualization of random walk simulations using both Matplotlib and Plotly. Random walks are fundamental stochastic processes that model how an object moves by taking random steps. They have applications in physics (Brownian motion), finance (stock prices), biology (molecular diffusion), and computer science (algorithms).

We'll explore different types of random walks including 1D, 2D, and multiple walker scenarios, showcasing various visualization techniques to understand the patterns and statistical properties of these processes.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import yaml
from pathlib import Path

# Import our custom modules
from src.generators.random_walk import RandomWalkGenerator
from src.plots.random_walk_mpl import RandomWalkMatplotlib
from src.plots.random_walk_plotly import RandomWalkPlotly
from src.utils.io import save_data, load_data
from src.utils.theming import get_plot_theme

# Load configuration
config_path = Path('config/random_walk.yaml')
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

print("Random Walk Configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")


# Initialize generator and plotting classes
rw_generator = RandomWalkGenerator(config)
mpl_plotter = RandomWalkMatplotlib(config)
plotly_plotter = RandomWalkPlotly(config)

# Set random seed for reproducibility
np.random.seed(config.get('random_seed', 42))

## Data Generation

In [ ]:
# Generate different types of random walks

# 1D Random Walk
steps_1d = rw_generator.generate_1d_walk(
    n_steps=config['n_steps'], 
    step_size=config['step_size']
)

# 2D Random Walk  
walk_2d = rw_generator.generate_2d_walk(
    n_steps=config['n_steps'],
    step_size=config['step_size']
)

# Multiple 1D walks for comparison
n_walks = config.get('n_walks', 5)
multiple_walks = []
for i in range(n_walks):
    walk = rw_generator.generate_1d_walk(
        n_steps=config['n_steps'],
        step_size=config['step_size']
    )
    multiple_walks.append(walk)

print(f"Generated:")
print(f"  - 1D walk with {len(steps_1d)} steps")
print(f"  - 2D walk with {len(walk_2d)} steps") 
print(f"  - {n_walks} multiple 1D walks for comparison")

# Save generated data
data_dir = Path('data/synthetic/random_walk')
data_dir.mkdir(parents=True, exist_ok=True)

save_data(steps_1d, data_dir / '1d_walk.csv')
save_data(walk_2d, data_dir / '2d_walk.csv')

# Save multiple walks
multiple_walks_df = pd.DataFrame({f'walk_{i+1}': walk for i, walk in enumerate(multiple_walks)})
save_data(multiple_walks_df, data_dir / 'multiple_walks.csv')

print("Data saved to data/synthetic/random_walk/")

## Matplotlib Visualizations

In [ ]:
# 1D Random Walk - Time Series
fig, ax = plt.subplots(figsize=(12, 6))
mpl_plotter.plot_1d_timeseries(steps_1d, ax=ax)
plt.title('1D Random Walk - Position vs Time')
plt.tight_layout()
plt.show()

# Save plot
exports_dir = Path('exports/images')
exports_dir.mkdir(parents=True, exist_ok=True)
plt.savefig(exports_dir / 'random_walk_1d_timeseries_mpl.png', dpi=300, bbox_inches='tight')


# 2D Random Walk - Path Visualization
fig, ax = plt.subplots(figsize=(10, 10))
mpl_plotter.plot_2d_path(walk_2d, ax=ax)
plt.title('2D Random Walk - Path Visualization')
plt.axis('equal')
plt.tight_layout()
plt.show()

plt.savefig(exports_dir / 'random_walk_2d_path_mpl.png', dpi=300, bbox_inches='tight')

# Multiple Random Walks Comparison
fig, ax = plt.subplots(figsize=(12, 8))
mpl_plotter.plot_multiple_walks(multiple_walks, ax=ax)
plt.title('Multiple Random Walks Comparison')
plt.tight_layout()
plt.show()

plt.savefig(exports_dir / 'random_walk_multiple_mpl.png', dpi=300, bbox_inches='tight')

# Statistical Analysis - Distribution of Final Positions
final_positions = [walk[-1] for walk in multiple_walks]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Histogram of final positions
ax1.hist(final_positions, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
ax1.set_xlabel('Final Position')
ax1.set_ylabel('Frequency')
ax1.set_title('Distribution of Final Positions')
ax1.grid(True, alpha=0.3)

# Cumulative distribution
sorted_positions = np.sort(final_positions)
cumulative = np.arange(1, len(sorted_positions) + 1) / len(sorted_positions)
ax2.plot(sorted_positions, cumulative, 'b-', linewidth=2)
ax2.set_xlabel('Final Position')
ax2.set_ylabel('Cumulative Probability')
ax2.set_title('Cumulative Distribution of Final Positions')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

plt.savefig(exports_dir / 'random_walk_statistics_mpl.png', dpi=300, bbox_inches='tight')

## Plotly Interactive Visualizations

In [ ]:
# Interactive 1D Random Walk
fig = plotly_plotter.plot_1d_interactive(steps_1d)
fig.update_layout(title="Interactive 1D Random Walk")
fig.show()

# Save as HTML
html_dir = Path('exports/html')
html_dir.mkdir(parents=True, exist_ok=True)
fig.write_html(html_dir / 'random_walk_1d_interactive.html')

# Interactive 2D Random Walk with Animation
fig = plotly_plotter.plot_2d_animated(walk_2d)
fig.update_layout(title="Animated 2D Random Walk")
fig.show()

fig.write_html(html_dir / 'random_walk_2d_animated.html')

# Multiple Walks Dashboard
fig = plotly_plotter.create_dashboard(multiple_walks, walk_2d)
fig.update_layout(title="Random Walk Analysis Dashboard")
fig.show()

fig.write_html(html_dir / 'random_walk_dashboard.html')

# 3D Random Walk (if implemented)
if hasattr(rw_generator, 'generate_3d_walk'):
    walk_3d = rw_generator.generate_3d_walk(
        n_steps=config['n_steps'],
        step_size=config['step_size']
    )
    
    fig = plotly_plotter.plot_3d_walk(walk_3d)
    fig.update_layout(title="3D Random Walk")
    fig.show()
    
    fig.write_html(html_dir / 'random_walk_3d.html')

## Advanced Analysis

In [ ]:
# Calculate statistical properties
def analyze_random_walk(walks_list):
    """Analyze statistical properties of random walks"""
    analysis = {}
    
    # Mean square displacement
    n_steps = len(walks_list[0])
    msd = np.zeros(n_steps)
    
    for walk in walks_list:
        cumulative_pos = np.cumsum(walk)
        msd += cumulative_pos ** 2
    
    msd /= len(walks_list)
    analysis['mean_square_displacement'] = msd
    
    # Final position statistics
    final_positions = [np.sum(walk) for walk in walks_list]
    analysis['final_pos_mean'] = np.mean(final_positions)
    analysis['final_pos_std'] = np.std(final_positions)
    analysis['final_pos_variance'] = np.var(final_positions)
    
    return analysis

# Perform analysis
analysis = analyze_random_walk(multiple_walks)

print("Random Walk Analysis:")
print(f"Final Position Statistics:")
print(f"  Mean: {analysis['final_pos_mean']:.3f}")
print(f"  Standard Deviation: {analysis['final_pos_std']:.3f}")
print(f"  Variance: {analysis['final_pos_variance']:.3f}")

# Plot Mean Square Displacement
fig, ax = plt.subplots(figsize=(10, 6))
time_steps = np.arange(len(analysis['mean_square_displacement']))
ax.plot(time_steps, analysis['mean_square_displacement'], 'b-', linewidth=2, label='MSD')

# Theoretical MSD for 1D random walk: σ²t where σ is step size std
theoretical_msd = config['step_size']**2 * time_steps
ax.plot(time_steps, theoretical_msd, 'r--', linewidth=2, label='Theoretical MSD')

ax.set_xlabel('Time Steps')
ax.set_ylabel('Mean Square Displacement')
ax.set_title('Mean Square Displacement Analysis')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

plt.savefig(exports_dir / 'random_walk_msd_analysis.png', dpi=300, bbox_inches='tight')

## Summary

This notebook successfully demonstrated the generation and visualization of random walk simulations using both Matplotlib and Plotly. Key findings and accomplishments include:

### Generated Data
- **1D Random Walk**: Single walker time series showing position evolution over time
- **2D Random Walk**: Spatial path visualization in two dimensions  
- **Multiple Walks**: Ensemble of random walks for statistical analysis

### Visualizations Created
- **Matplotlib**: Static plots including time series, 2D paths, multiple walk comparisons, and statistical distributions
- **Plotly**: Interactive and animated visualizations with dashboards for comprehensive analysis

### Key Insights
- Random walks exhibit the expected √t scaling behavior in mean square displacement
- Final position distributions follow theoretical predictions for unbiased random walks
- 2D random walks create complex, self-intersecting paths that never truly "escape" their origin
- Multiple walk ensembles reveal the statistical nature of the random process

### Technical Implementation
- Modular design with separate generator and plotting classes
- Configuration-driven approach for reproducibility
- Both static and interactive visualization capabilities
- Proper data export for further analysis

The random walk simulations provide an excellent foundation for understanding stochastic processes and serve as building blocks for more complex models in physics, finance, and other domains requiring probabilistic modeling.

All visualizations have been exported to the `exports/` directory for documentation and presentation purposes.